In [1]:
#imports
import random
import calendar
import pandas as pd

In [2]:
#Random year & month
year = random.randint(1970,2100)
month = random.randint(1,12)
print("Year :" , year," Month: ",month)

#no of days
start_day, no_of_days = calendar.monthrange(year, month)
start_day = (calendar.day_name[start_day])
print("No. of days: ", no_of_days)
print("Starting day of the week: ", start_day)

Year : 2039  Month:  1
No. of days:  31
Starting day of the week:  Saturday


In [3]:
def create_date_table(start, end):
    df = pd.DataFrame({"Date": pd.date_range(start, end)})
    df["Week"] = df.Date.dt.weekofyear
    df["Day"] = df.Date.dt.weekday_name
    return df

In [4]:
df = create_date_table(str(year) + '-' + str(month) +'-01', end=str(year) + '-' + str(month) +'-'+str(no_of_days))


In [5]:
df

,Date,Week,Day
0,2039-01-01,52,Saturday
1,2039-01-02,52,Sunday
2,2039-01-03,1,Monday
3,2039-01-04,1,Tuesday
4,2039-01-05,1,Wednesday
5,2039-01-06,1,Thursday
6,2039-01-07,1,Friday
7,2039-01-08,1,Saturday
8,2039-01-09,1,Sunday
9,2039-01-10,2,Monday
